# ADX Start

In [ ]:
# trading_system/scripts/run_ao_optimization.py
"""
Main script to run hyperparameter optimization and sensitivity analysis
for the Awesome Oscillator strategy using the portfolio-based evaluation framework.
"""
import json
import logging
import os
import sys
from datetime import datetime, timedelta

import mlflow
import pandas as pd

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

try:
    from src.strategies.trend_following.adx_strat import ADXStrategy
    from src.optimizer.strategy_optimizer import StrategyOptimizer
    from src.optimizer.sensitivity_analyzer import SensitivityAnalyzer
    from src.database.config import DatabaseConfig
    from utils.file_utils import load_tickers_from_yaml
except ImportError as e:
    print("Error importing modules. Make sure the script is run from the project root")
    print("or the 'src' directory is in the Python path.")
    print(f"Import Error: {e}")
    sys.exit(1)

In [ ]:
# single stock

# Logging Configuration
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger(__name__)

# MLflow Configuration
MLFLOW_TRACKING_URI = "file:./mlruns"  # Store MLflow data locally in ./mlruns
RUN_TIMESTAMP = datetime.now().strftime("%Y%m%d_%H%M%S")

# Data Configuration
TICKER_FILE_PATH = "../data/tickers.yml" # Path relative to project root
MAX_TICKERS = None # Limit tickers for faster testing, set to None to use all

# Backtest Period
START_DATE = (datetime.now() - timedelta(days=4*365)).strftime("%Y-%m-%d")
END_DATE = datetime.now().strftime("%Y-%m-%d")

# Optimization Settings
CV_FOLDS = 5
MAX_EVALS = 50  # Number of hyperparameter sets to evaluate
OPTIMIZATION_METRIC = 'harmonic_mean' # Portfolio metric to maximize (minus penalty)
N_JOBS = -1 # Use all available CPU cores for fold evaluation within optimizer

# Sensitivity Analysis Settings
RUN_SENSITIVITY = False # Set to False to skip sensitivity analysis
NUMERIC_PERTURBATION = 0.15 # +/- 15% for sensitivity
SENS_SAMPLES_PER_PARAM = 5
SENS_RANDOM_SAMPLES = 20

# --- Define Search Space for Awesome Oscillator ---

# Note: Hyperopt doesn't easily enforce short_period < long_period directly during sampling.
# The optimizer will evaluate invalid combinations, and they will likely fail or perform poorly.
# Strategy itself raises ValueError if short >= long during initialization.

from src.optimizer.search_space import adx_strat_search_space

# --- Main Execution ---

if __name__ == "__main__":
    logger.info("--- Starting Awesome Oscillator Optimization Script ---")

    # Setup MLflow
    try:
        mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
        logger.info(f"MLflow tracking URI set to: {MLFLOW_TRACKING_URI}")
    except Exception as e:
        logger.error(f"Failed to set MLflow tracking URI: {e}")
        sys.exit(1)

        # Create MLflow experiment if it doesn't exist
    try:
        experiment_name = f"ADX_Start_{RUN_TIMESTAMP}"
        # Check if experiment exists
        experiment = mlflow.get_experiment_by_name(experiment_name)
        if experiment is None:
            # Create new experiment
            experiment_id = mlflow.create_experiment(experiment_name)
            logger.info(f"Created new MLflow experiment: {experiment_name} with ID: {experiment_id}")
        else:
            experiment_id = experiment.experiment_id
            logger.info(f"Using existing MLflow experiment: {experiment_name} with ID: {experiment_id}")
        
        # Set the experiment for subsequent runs
        mlflow.set_experiment(experiment_name)
    except Exception as e:
        logger.error(f"Failed to create or set MLflow experiment: {e}")
        sys.exit(1)

    # Load Tickers
    try:
        tickers_to_run = load_tickers_from_yaml(TICKER_FILE_PATH, MAX_TICKERS)
    except Exception:
        logger.error("Failed to load tickers. Exiting.")
        sys.exit(1)

    # Database Config
    try:
        db_config = DatabaseConfig.default()
        # Optional: Add a check here to ensure DB connection is valid if possible
        logger.info("Database configuration loaded.")
    except Exception as e:
        logger.error(f"Failed to load database configuration: {e}")
        sys.exit(1)

    # --- Run Optimization ---
    optimizer = None
    best_params = {}
    portfolio_performance_report = pd.DataFrame()
    param_history_report = pd.DataFrame()

    logger.info(f"Initializing StrategyOptimizer for {ADXStrategy.__name__}")
    try:
        optimizer = StrategyOptimizer(
            strategy_class=ADXStrategy,
            db_config=db_config,
            search_space=adx_strat_search_space,
            tickers=tickers_to_run,
            start_date=START_DATE,
            end_date=END_DATE,
            cv_folds=CV_FOLDS,
            max_evals=MAX_EVALS,
            optimization_metric=OPTIMIZATION_METRIC,
            run_name=f"AwesomeOscillator_Opt_{RUN_TIMESTAMP}",
            n_jobs=N_JOBS
            # risk_thresholds can be customized here if needed, otherwise defaults are used
        )

        logger.info("Starting hyperparameter optimization...")
        best_params, portfolio_performance_report, param_history_report = optimizer.run_optimization()

        if not best_params:
             logger.error("Optimization did not yield valid results. Best parameters not found.")
        else:
             logger.info("--- Optimization Results ---")
             logger.info(f"Best Parameters found:\n{json.dumps(best_params, indent=2)}")
             logger.info(f"\nBest Portfolio Performance Report:\n{portfolio_performance_report.to_string()}")
             logger.info(f"\nParameter History saved (see MLflow artifacts or CSV file). Head:\n{param_history_report.head().to_string()}")

    except Exception as e:
        logger.error(f"An error occurred during optimization: {e}", exc_info=True)
        # Attempt to end MLflow run if it was started by the optimizer
        if mlflow.active_run():
            mlflow.end_run("FAILED")

    # --- Run Sensitivity Analysis (Optional) ---
    if RUN_SENSITIVITY and optimizer and best_params:
        logger.info("\n--- Starting Sensitivity Analysis ---")
        try:
            analyzer = SensitivityAnalyzer(
                strategy_optimizer=optimizer, # Reuse optimizer for its config and evaluation cache
                base_params=best_params,
                numeric_perturbation=NUMERIC_PERTURBATION,
                num_samples_per_param=SENS_SAMPLES_PER_PARAM,
                num_random_samples=SENS_RANDOM_SAMPLES,
                parallel=True # Relies on optimizer's internal parallelization/caching
            )

            sensitivity_results_df, parameter_impact_df = analyzer.run()

            if sensitivity_results_df.empty:
                 logger.warning("Sensitivity analysis did not produce results.")
            else:
                logger.info("--- Sensitivity Analysis Results ---")
                logger.info(f"Sensitivity Results saved (see MLflow artifacts or CSV file). Head:\n{sensitivity_results_df.head().to_string()}")
                logger.info(f"\nParameter Impact Report (Correlation):\n{parameter_impact_df.to_string()}")

        except Exception as e:
            logger.error(f"An error occurred during sensitivity analysis: {e}", exc_info=True)
            if mlflow.active_run():
                 mlflow.end_run("FAILED") # End sensitivity run if it crashed

    elif RUN_SENSITIVITY and (not optimizer or not best_params):
        logger.warning("Skipping sensitivity analysis because optimization failed or produced no best parameters.")


    # Ensure any lingering run is terminated cleanly
    # Should not be necessary if 'with mlflow.start_run()' is used correctly inside modules
    # try:
    #     while mlflow.active_run():
    #         logger.info(f"Ending lingering MLflow run: {mlflow.active_run().info.run_id}")
    #         mlflow.end_run()
    # except Exception:
    #      pass # Ignore errors during cleanup

    logger.info("--- Script Finished ---")

# Aroon Strategy

In [ ]:
# trading_system/scripts/run_ao_optimization.py
"""
Main script to run hyperparameter optimization and sensitivity analysis
for the Awesome Oscillator strategy using the portfolio-based evaluation framework.
"""
import json
import logging
import os
import sys
from datetime import datetime, timedelta

import mlflow
import pandas as pd

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

try:
    from src.strategies.trend_following.aroon_strat import AroonStrategy
    from src.optimizer.strategy_optimizer import StrategyOptimizer
    from src.optimizer.sensitivity_analyzer import SensitivityAnalyzer
    from src.database.config import DatabaseConfig
    from utils.file_utils import load_tickers_from_yaml
except ImportError as e:
    print("Error importing modules. Make sure the script is run from the project root")
    print("or the 'src' directory is in the Python path.")
    print(f"Import Error: {e}")
    sys.exit(1)


In [ ]:
# single stock

# Logging Configuration
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger(__name__)

# MLflow Configuration
MLFLOW_TRACKING_URI = "file:./mlruns"  # Store MLflow data locally in ./mlruns
RUN_TIMESTAMP = datetime.now().strftime("%Y%m%d_%H%M%S")

# Data Configuration
TICKER_FILE_PATH = "../data/tickers.yml" # Path relative to project root
MAX_TICKERS = None # Limit tickers for faster testing, set to None to use all


# Backtest Period
START_DATE = (datetime.now() - timedelta(days=4*365)).strftime("%Y-%m-%d")
END_DATE = datetime.now().strftime("%Y-%m-%d")

# Optimization Settings
CV_FOLDS = 5
MAX_EVALS = 50  # Number of hyperparameter sets to evaluate
OPTIMIZATION_METRIC = 'harmonic_mean' # Portfolio metric to maximize (minus penalty)
N_JOBS = -1 # Use all available CPU cores for fold evaluation within optimizer

# Sensitivity Analysis Settings
RUN_SENSITIVITY = False # Set to False to skip sensitivity analysis
NUMERIC_PERTURBATION = 0.15 # +/- 15% for sensitivity
SENS_SAMPLES_PER_PARAM = 5
SENS_RANDOM_SAMPLES = 20

# --- Define Search Space for Awesome Oscillator ---

# Note: Hyperopt doesn't easily enforce short_period < long_period directly during sampling.
# The optimizer will evaluate invalid combinations, and they will likely fail or perform poorly.
# Strategy itself raises ValueError if short >= long during initialization.

from src.optimizer.search_space import aroon_strat_search_space

# --- Main Execution ---

if __name__ == "__main__":
    logger.info("--- Starting Awesome Oscillator Optimization Script ---")

    # Setup MLflow
    try:
        mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
        logger.info(f"MLflow tracking URI set to: {MLFLOW_TRACKING_URI}")
    except Exception as e:
        logger.error(f"Failed to set MLflow tracking URI: {e}")
        sys.exit(1)

        # Create MLflow experiment if it doesn't exist
    try:
        experiment_name = f"ADX_Start_{RUN_TIMESTAMP}"
        # Check if experiment exists
        experiment = mlflow.get_experiment_by_name(experiment_name)
        if experiment is None:
            # Create new experiment
            experiment_id = mlflow.create_experiment(experiment_name)
            logger.info(f"Created new MLflow experiment: {experiment_name} with ID: {experiment_id}")
        else:
            experiment_id = experiment.experiment_id
            logger.info(f"Using existing MLflow experiment: {experiment_name} with ID: {experiment_id}")
        
        # Set the experiment for subsequent runs
        mlflow.set_experiment(experiment_name)
    except Exception as e:
        logger.error(f"Failed to create or set MLflow experiment: {e}")
        sys.exit(1)

    # Load Tickers
    try:
        tickers_to_run = load_tickers_from_yaml(TICKER_FILE_PATH, MAX_TICKERS)
    except Exception:
        logger.error("Failed to load tickers. Exiting.")
        sys.exit(1)

    # Database Config
    try:
        db_config = DatabaseConfig.default()
        # Optional: Add a check here to ensure DB connection is valid if possible
        logger.info("Database configuration loaded.")
    except Exception as e:
        logger.error(f"Failed to load database configuration: {e}")
        sys.exit(1)

    # --- Run Optimization ---
    optimizer = None
    best_params = {}
    portfolio_performance_report = pd.DataFrame()
    param_history_report = pd.DataFrame()

    logger.info(f"Initializing StrategyOptimizer for {AroonStrategy.__name__}")
    try:
        optimizer = StrategyOptimizer(
            strategy_class=AroonStrategy,
            db_config=db_config,
            search_space=aroon_strat_search_space,
            tickers=tickers_to_run,
            start_date=START_DATE,
            end_date=END_DATE,
            cv_folds=CV_FOLDS,
            max_evals=MAX_EVALS,
            optimization_metric=OPTIMIZATION_METRIC,
            run_name=f"AwesomeOscillator_Opt_{RUN_TIMESTAMP}",
            n_jobs=N_JOBS
            # risk_thresholds can be customized here if needed, otherwise defaults are used
        )

        logger.info("Starting hyperparameter optimization...")
        best_params, portfolio_performance_report, param_history_report = optimizer.run_optimization()

        if not best_params:
             logger.error("Optimization did not yield valid results. Best parameters not found.")
        else:
             logger.info("--- Optimization Results ---")
             logger.info(f"Best Parameters found:\n{json.dumps(best_params, indent=2)}")
             logger.info(f"\nBest Portfolio Performance Report:\n{portfolio_performance_report.to_string()}")
             logger.info(f"\nParameter History saved (see MLflow artifacts or CSV file). Head:\n{param_history_report.head().to_string()}")

    except Exception as e:
        logger.error(f"An error occurred during optimization: {e}", exc_info=True)
        # Attempt to end MLflow run if it was started by the optimizer
        if mlflow.active_run():
            mlflow.end_run("FAILED")

    # --- Run Sensitivity Analysis (Optional) ---
    if RUN_SENSITIVITY and optimizer and best_params:
        logger.info("\n--- Starting Sensitivity Analysis ---")
        try:
            analyzer = SensitivityAnalyzer(
                strategy_optimizer=optimizer, # Reuse optimizer for its config and evaluation cache
                base_params=best_params,
                numeric_perturbation=NUMERIC_PERTURBATION,
                num_samples_per_param=SENS_SAMPLES_PER_PARAM,
                num_random_samples=SENS_RANDOM_SAMPLES,
                parallel=True # Relies on optimizer's internal parallelization/caching
            )

            sensitivity_results_df, parameter_impact_df = analyzer.run()

            if sensitivity_results_df.empty:
                 logger.warning("Sensitivity analysis did not produce results.")
            else:
                logger.info("--- Sensitivity Analysis Results ---")
                logger.info(f"Sensitivity Results saved (see MLflow artifacts or CSV file). Head:\n{sensitivity_results_df.head().to_string()}")
                logger.info(f"\nParameter Impact Report (Correlation):\n{parameter_impact_df.to_string()}")

        except Exception as e:
            logger.error(f"An error occurred during sensitivity analysis: {e}", exc_info=True)
            if mlflow.active_run():
                 mlflow.end_run("FAILED") # End sensitivity run if it crashed

    elif RUN_SENSITIVITY and (not optimizer or not best_params):
        logger.warning("Skipping sensitivity analysis because optimization failed or produced no best parameters.")


    # Ensure any lingering run is terminated cleanly
    # Should not be necessary if 'with mlflow.start_run()' is used correctly inside modules
    # try:
    #     while mlflow.active_run():
    #         logger.info(f"Ending lingering MLflow run: {mlflow.active_run().info.run_id}")
    #         mlflow.end_run()
    # except Exception:
    #      pass # Ignore errors during cleanup

    logger.info("--- Script Finished ---")

# Choppiness Index

In [ ]:
# trading_system/scripts/run_ao_optimization.py
"""
Main script to run hyperparameter optimization and sensitivity analysis
for the Awesome Oscillator strategy using the portfolio-based evaluation framework.
"""
import json
import logging
import os
import sys
from datetime import datetime, timedelta

import mlflow
import pandas as pd

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

try:
    from src.strategies.trend_following.choppiness_index import ChoppinessIndexStrategy
    from src.optimizer.strategy_optimizer import StrategyOptimizer
    from src.optimizer.sensitivity_analyzer import SensitivityAnalyzer
    from src.database.config import DatabaseConfig
    from utils.file_utils import load_tickers_from_yaml
except ImportError as e:
    print("Error importing modules. Make sure the script is run from the project root")
    print("or the 'src' directory is in the Python path.")
    print(f"Import Error: {e}")
    sys.exit(1)


In [ ]:
# single stock

# Logging Configuration
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger(__name__)

# MLflow Configuration
MLFLOW_TRACKING_URI = "file:./mlruns"  # Store MLflow data locally in ./mlruns
RUN_TIMESTAMP = datetime.now().strftime("%Y%m%d_%H%M%S")

# Data Configuration
TICKER_FILE_PATH = "../data/tickers.yml" # Path relative to project root
MAX_TICKERS = None # Limit tickers for faster testing, set to None to use all


# Backtest Period
START_DATE = (datetime.now() - timedelta(days=4*365)).strftime("%Y-%m-%d")
END_DATE = datetime.now().strftime("%Y-%m-%d")

# Optimization Settings
CV_FOLDS = 5
MAX_EVALS = 50  # Number of hyperparameter sets to evaluate
OPTIMIZATION_METRIC = 'harmonic_mean' # Portfolio metric to maximize (minus penalty)
N_JOBS = -1 # Use all available CPU cores for fold evaluation within optimizer

# Sensitivity Analysis Settings
RUN_SENSITIVITY = False # Set to False to skip sensitivity analysis
NUMERIC_PERTURBATION = 0.15 # +/- 15% for sensitivity
SENS_SAMPLES_PER_PARAM = 5
SENS_RANDOM_SAMPLES = 20

# --- Define Search Space for Awesome Oscillator ---

# Note: Hyperopt doesn't easily enforce short_period < long_period directly during sampling.
# The optimizer will evaluate invalid combinations, and they will likely fail or perform poorly.
# Strategy itself raises ValueError if short >= long during initialization.

from src.optimizer.search_space import choppiness_index_strat_search_space

# --- Main Execution ---

if __name__ == "__main__":
    logger.info("--- Starting Awesome Oscillator Optimization Script ---")

    # Setup MLflow
    try:
        mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
        logger.info(f"MLflow tracking URI set to: {MLFLOW_TRACKING_URI}")
    except Exception as e:
        logger.error(f"Failed to set MLflow tracking URI: {e}")
        sys.exit(1)

        # Create MLflow experiment if it doesn't exist
    try:
        experiment_name = f"ADX_Start_{RUN_TIMESTAMP}"
        # Check if experiment exists
        experiment = mlflow.get_experiment_by_name(experiment_name)
        if experiment is None:
            # Create new experiment
            experiment_id = mlflow.create_experiment(experiment_name)
            logger.info(f"Created new MLflow experiment: {experiment_name} with ID: {experiment_id}")
        else:
            experiment_id = experiment.experiment_id
            logger.info(f"Using existing MLflow experiment: {experiment_name} with ID: {experiment_id}")
        
        # Set the experiment for subsequent runs
        mlflow.set_experiment(experiment_name)
    except Exception as e:
        logger.error(f"Failed to create or set MLflow experiment: {e}")
        sys.exit(1)

    # Load Tickers
    try:
        tickers_to_run = load_tickers_from_yaml(TICKER_FILE_PATH, MAX_TICKERS)
    except Exception:
        logger.error("Failed to load tickers. Exiting.")
        sys.exit(1)

    # Database Config
    try:
        db_config = DatabaseConfig.default()
        # Optional: Add a check here to ensure DB connection is valid if possible
        logger.info("Database configuration loaded.")
    except Exception as e:
        logger.error(f"Failed to load database configuration: {e}")
        sys.exit(1)

    # --- Run Optimization ---
    optimizer = None
    best_params = {}
    portfolio_performance_report = pd.DataFrame()
    param_history_report = pd.DataFrame()

    logger.info(f"Initializing StrategyOptimizer for {ChoppinessIndexStrategy.__name__}")
    try:
        optimizer = StrategyOptimizer(
            strategy_class=ChoppinessIndexStrategy,
            db_config=db_config,
            search_space=choppiness_index_strat_search_space,
            tickers=tickers_to_run,
            start_date=START_DATE,
            end_date=END_DATE,
            cv_folds=CV_FOLDS,
            max_evals=MAX_EVALS,
            optimization_metric=OPTIMIZATION_METRIC,
            run_name=f"Choppiness_index_Opt_{RUN_TIMESTAMP}",
            n_jobs=N_JOBS
            # risk_thresholds can be customized here if needed, otherwise defaults are used
        )

        logger.info("Starting hyperparameter optimization...")
        best_params, portfolio_performance_report, param_history_report = optimizer.run_optimization()

        if not best_params:
             logger.error("Optimization did not yield valid results. Best parameters not found.")
        else:
             logger.info("--- Optimization Results ---")
             logger.info(f"Best Parameters found:\n{json.dumps(best_params, indent=2)}")
             logger.info(f"\nBest Portfolio Performance Report:\n{portfolio_performance_report.to_string()}")
             logger.info(f"\nParameter History saved (see MLflow artifacts or CSV file). Head:\n{param_history_report.head().to_string()}")

    except Exception as e:
        logger.error(f"An error occurred during optimization: {e}", exc_info=True)
        # Attempt to end MLflow run if it was started by the optimizer
        if mlflow.active_run():
            mlflow.end_run("FAILED")

    # --- Run Sensitivity Analysis (Optional) ---
    if RUN_SENSITIVITY and optimizer and best_params:
        logger.info("\n--- Starting Sensitivity Analysis ---")
        try:
            analyzer = SensitivityAnalyzer(
                strategy_optimizer=optimizer, # Reuse optimizer for its config and evaluation cache
                base_params=best_params,
                numeric_perturbation=NUMERIC_PERTURBATION,
                num_samples_per_param=SENS_SAMPLES_PER_PARAM,
                num_random_samples=SENS_RANDOM_SAMPLES,
                parallel=True # Relies on optimizer's internal parallelization/caching
            )

            sensitivity_results_df, parameter_impact_df = analyzer.run()

            if sensitivity_results_df.empty:
                 logger.warning("Sensitivity analysis did not produce results.")
            else:
                logger.info("--- Sensitivity Analysis Results ---")
                logger.info(f"Sensitivity Results saved (see MLflow artifacts or CSV file). Head:\n{sensitivity_results_df.head().to_string()}")
                logger.info(f"\nParameter Impact Report (Correlation):\n{parameter_impact_df.to_string()}")

        except Exception as e:
            logger.error(f"An error occurred during sensitivity analysis: {e}", exc_info=True)
            if mlflow.active_run():
                 mlflow.end_run("FAILED") # End sensitivity run if it crashed

    elif RUN_SENSITIVITY and (not optimizer or not best_params):
        logger.warning("Skipping sensitivity analysis because optimization failed or produced no best parameters.")


    # Ensure any lingering run is terminated cleanly
    # Should not be necessary if 'with mlflow.start_run()' is used correctly inside modules
    # try:
    #     while mlflow.active_run():
    #         logger.info(f"Ending lingering MLflow run: {mlflow.active_run().info.run_id}")
    #         mlflow.end_run()
    # except Exception:
    #      pass # Ignore errors during cleanup

    logger.info("--- Script Finished ---")

# donchian Channel

In [ ]:
# trading_system/scripts/run_ao_optimization.py
"""
Main script to run hyperparameter optimization and sensitivity analysis
for the Awesome Oscillator strategy using the portfolio-based evaluation framework.
"""
import json
import logging
import os
import sys
from datetime import datetime, timedelta

import mlflow
import pandas as pd

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

try:
    from src.strategies.trend_following.donchian_channel_strat import DonchianChannelStrategy
    from src.optimizer.strategy_optimizer import StrategyOptimizer
    from src.optimizer.sensitivity_analyzer import SensitivityAnalyzer
    from src.database.config import DatabaseConfig
    from utils.file_utils import load_tickers_from_yaml
except ImportError as e:
    print("Error importing modules. Make sure the script is run from the project root")
    print("or the 'src' directory is in the Python path.")
    print(f"Import Error: {e}")
    sys.exit(1)


In [ ]:
# single stock

# Logging Configuration
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger(__name__)

# MLflow Configuration
MLFLOW_TRACKING_URI = "file:./mlruns"  # Store MLflow data locally in ./mlruns
RUN_TIMESTAMP = datetime.now().strftime("%Y%m%d_%H%M%S")

# Data Configuration
TICKER_FILE_PATH = "../data/tickers.yml" # Path relative to project root
MAX_TICKERS = None # Limit tickers for faster testing, set to None to use all

# Backtest Period
START_DATE = (datetime.now() - timedelta(days=4*365)).strftime("%Y-%m-%d")
END_DATE = datetime.now().strftime("%Y-%m-%d")

# Optimization Settings
CV_FOLDS = 5
MAX_EVALS = 50  # Number of hyperparameter sets to evaluate
OPTIMIZATION_METRIC = 'harmonic_mean' # Portfolio metric to maximize (minus penalty)
N_JOBS = -1 # Use all available CPU cores for fold evaluation within optimizer

# Sensitivity Analysis Settings
RUN_SENSITIVITY = False # Set to False to skip sensitivity analysis
NUMERIC_PERTURBATION = 0.15 # +/- 15% for sensitivity
SENS_SAMPLES_PER_PARAM = 5
SENS_RANDOM_SAMPLES = 20

# --- Define Search Space for Awesome Oscillator ---

# Note: Hyperopt doesn't easily enforce short_period < long_period directly during sampling.
# The optimizer will evaluate invalid combinations, and they will likely fail or perform poorly.
# Strategy itself raises ValueError if short >= long during initialization.

from src.optimizer.search_space import donchian_channel_strat_search_space

# --- Main Execution ---

if __name__ == "__main__":
    logger.info("--- Starting Awesome Oscillator Optimization Script ---")

    # Setup MLflow
    try:
        mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
        logger.info(f"MLflow tracking URI set to: {MLFLOW_TRACKING_URI}")
    except Exception as e:
        logger.error(f"Failed to set MLflow tracking URI: {e}")
        sys.exit(1)

        # Create MLflow experiment if it doesn't exist
    try:
        experiment_name = f"Donchian_{RUN_TIMESTAMP}"
        # Check if experiment exists
        experiment = mlflow.get_experiment_by_name(experiment_name)
        if experiment is None:
            # Create new experiment
            experiment_id = mlflow.create_experiment(experiment_name)
            logger.info(f"Created new MLflow experiment: {experiment_name} with ID: {experiment_id}")
        else:
            experiment_id = experiment.experiment_id
            logger.info(f"Using existing MLflow experiment: {experiment_name} with ID: {experiment_id}")
        
        # Set the experiment for subsequent runs
        mlflow.set_experiment(experiment_name)
    except Exception as e:
        logger.error(f"Failed to create or set MLflow experiment: {e}")
        sys.exit(1)

    # Load Tickers
    try:
        tickers_to_run = load_tickers_from_yaml(TICKER_FILE_PATH, MAX_TICKERS)
    except Exception:
        logger.error("Failed to load tickers. Exiting.")
        sys.exit(1)

    # Database Config
    try:
        db_config = DatabaseConfig.default()
        # Optional: Add a check here to ensure DB connection is valid if possible
        logger.info("Database configuration loaded.")
    except Exception as e:
        logger.error(f"Failed to load database configuration: {e}")
        sys.exit(1)

    # --- Run Optimization ---
    optimizer = None
    best_params = {}
    portfolio_performance_report = pd.DataFrame()
    param_history_report = pd.DataFrame()

    logger.info(f"Initializing StrategyOptimizer for {DonchianChannelStrategy.__name__}")
    try:
        optimizer = StrategyOptimizer(
            strategy_class=DonchianChannelStrategy,
            db_config=db_config,
            search_space=donchian_channel_strat_search_space,
            tickers=tickers_to_run,
            start_date=START_DATE,
            end_date=END_DATE,
            cv_folds=CV_FOLDS,
            max_evals=MAX_EVALS,
            optimization_metric=OPTIMIZATION_METRIC,
            run_name=f"Choppiness_index_Opt_{RUN_TIMESTAMP}",
            n_jobs=N_JOBS
            # risk_thresholds can be customized here if needed, otherwise defaults are used
        )

        logger.info("Starting hyperparameter optimization...")
        best_params, portfolio_performance_report, param_history_report = optimizer.run_optimization()

        if not best_params:
             logger.error("Optimization did not yield valid results. Best parameters not found.")
        else:
             logger.info("--- Optimization Results ---")
             logger.info(f"Best Parameters found:\n{json.dumps(best_params, indent=2)}")
             logger.info(f"\nBest Portfolio Performance Report:\n{portfolio_performance_report.to_string()}")
             logger.info(f"\nParameter History saved (see MLflow artifacts or CSV file). Head:\n{param_history_report.head().to_string()}")

    except Exception as e:
        logger.error(f"An error occurred during optimization: {e}", exc_info=True)
        # Attempt to end MLflow run if it was started by the optimizer
        if mlflow.active_run():
            mlflow.end_run("FAILED")

    # --- Run Sensitivity Analysis (Optional) ---
    if RUN_SENSITIVITY and optimizer and best_params:
        logger.info("\n--- Starting Sensitivity Analysis ---")
        try:
            analyzer = SensitivityAnalyzer(
                strategy_optimizer=optimizer, # Reuse optimizer for its config and evaluation cache
                base_params=best_params,
                numeric_perturbation=NUMERIC_PERTURBATION,
                num_samples_per_param=SENS_SAMPLES_PER_PARAM,
                num_random_samples=SENS_RANDOM_SAMPLES,
                parallel=True # Relies on optimizer's internal parallelization/caching
            )

            sensitivity_results_df, parameter_impact_df = analyzer.run()

            if sensitivity_results_df.empty:
                 logger.warning("Sensitivity analysis did not produce results.")
            else:
                logger.info("--- Sensitivity Analysis Results ---")
                logger.info(f"Sensitivity Results saved (see MLflow artifacts or CSV file). Head:\n{sensitivity_results_df.head().to_string()}")
                logger.info(f"\nParameter Impact Report (Correlation):\n{parameter_impact_df.to_string()}")

        except Exception as e:
            logger.error(f"An error occurred during sensitivity analysis: {e}", exc_info=True)
            if mlflow.active_run():
                 mlflow.end_run("FAILED") # End sensitivity run if it crashed

    elif RUN_SENSITIVITY and (not optimizer or not best_params):
        logger.warning("Skipping sensitivity analysis because optimization failed or produced no best parameters.")


    # Ensure any lingering run is terminated cleanly
    # Should not be necessary if 'with mlflow.start_run()' is used correctly inside modules
    # try:
    #     while mlflow.active_run():
    #         logger.info(f"Ending lingering MLflow run: {mlflow.active_run().info.run_id}")
    #         mlflow.end_run()
    # except Exception:
    #      pass # Ignore errors during cleanup

    logger.info("--- Script Finished ---")

# Ichimoku Cloud

In [ ]:
# trading_system/scripts/run_ao_optimization.py
"""
Main script to run hyperparameter optimization and sensitivity analysis
for the Awesome Oscillator strategy using the portfolio-based evaluation framework.
"""
import json
import logging
import os
import sys
from datetime import datetime, timedelta

import mlflow
import pandas as pd

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

try:
    from src.strategies.trend_following.ichimoku_cloud_strat import IchimokuCloudStrategy
    from src.optimizer.strategy_optimizer import StrategyOptimizer
    from src.optimizer.sensitivity_analyzer import SensitivityAnalyzer
    from src.database.config import DatabaseConfig
    from utils.file_utils import load_tickers_from_yaml
except ImportError as e:
    print("Error importing modules. Make sure the script is run from the project root")
    print("or the 'src' directory is in the Python path.")
    print(f"Import Error: {e}")
    sys.exit(1)


In [ ]:
# single stock

# Logging Configuration
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger(__name__)

# MLflow Configuration
MLFLOW_TRACKING_URI = "file:./mlruns"  # Store MLflow data locally in ./mlruns
RUN_TIMESTAMP = datetime.now().strftime("%Y%m%d_%H%M%S")

# Data Configuration
TICKER_FILE_PATH = "../data/tickers.yml" # Path relative to project root
MAX_TICKERS = None # Limit tickers for faster testing, set to None to use all


# Backtest Period
START_DATE = (datetime.now() - timedelta(days=4*365)).strftime("%Y-%m-%d")
END_DATE = datetime.now().strftime("%Y-%m-%d")

# Optimization Settings
CV_FOLDS = 5
MAX_EVALS = 50  # Number of hyperparameter sets to evaluate
OPTIMIZATION_METRIC = 'harmonic_mean' # Portfolio metric to maximize (minus penalty)
N_JOBS = -1 # Use all available CPU cores for fold evaluation within optimizer

# Sensitivity Analysis Settings
RUN_SENSITIVITY = False # Set to False to skip sensitivity analysis
NUMERIC_PERTURBATION = 0.15 # +/- 15% for sensitivity
SENS_SAMPLES_PER_PARAM = 5
SENS_RANDOM_SAMPLES = 20

# --- Define Search Space for Awesome Oscillator ---

# Note: Hyperopt doesn't easily enforce short_period < long_period directly during sampling.
# The optimizer will evaluate invalid combinations, and they will likely fail or perform poorly.
# Strategy itself raises ValueError if short >= long during initialization.

from src.optimizer.search_space import ichimoku_cloud_strat_search_space

# --- Main Execution ---

if __name__ == "__main__":
    logger.info("--- Starting Awesome Oscillator Optimization Script ---")

    # Setup MLflow
    try:
        mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
        logger.info(f"MLflow tracking URI set to: {MLFLOW_TRACKING_URI}")
    except Exception as e:
        logger.error(f"Failed to set MLflow tracking URI: {e}")
        sys.exit(1)

        # Create MLflow experiment if it doesn't exist
    try:
        experiment_name = f"Donchian_{RUN_TIMESTAMP}"
        # Check if experiment exists
        experiment = mlflow.get_experiment_by_name(experiment_name)
        if experiment is None:
            # Create new experiment
            experiment_id = mlflow.create_experiment(experiment_name)
            logger.info(f"Created new MLflow experiment: {experiment_name} with ID: {experiment_id}")
        else:
            experiment_id = experiment.experiment_id
            logger.info(f"Using existing MLflow experiment: {experiment_name} with ID: {experiment_id}")
        
        # Set the experiment for subsequent runs
        mlflow.set_experiment(experiment_name)
    except Exception as e:
        logger.error(f"Failed to create or set MLflow experiment: {e}")
        sys.exit(1)

    # Load Tickers
    try:
        tickers_to_run = load_tickers_from_yaml(TICKER_FILE_PATH, MAX_TICKERS)
    except Exception:
        logger.error("Failed to load tickers. Exiting.")
        sys.exit(1)

    # Database Config
    try:
        db_config = DatabaseConfig.default()
        # Optional: Add a check here to ensure DB connection is valid if possible
        logger.info("Database configuration loaded.")
    except Exception as e:
        logger.error(f"Failed to load database configuration: {e}")
        sys.exit(1)

    # --- Run Optimization ---
    optimizer = None
    best_params = {}
    portfolio_performance_report = pd.DataFrame()
    param_history_report = pd.DataFrame()

    logger.info(f"Initializing StrategyOptimizer for {IchimokuCloudStrategy.__name__}")
    try:
        optimizer = StrategyOptimizer(
            strategy_class=IchimokuCloudStrategy,
            db_config=db_config,
            search_space=ichimoku_cloud_strat_search_space,
            tickers=tickers_to_run,
            start_date=START_DATE,
            end_date=END_DATE,
            cv_folds=CV_FOLDS,
            max_evals=MAX_EVALS,
            optimization_metric=OPTIMIZATION_METRIC,
            run_name=f"Ichimoku_Opt_{RUN_TIMESTAMP}",
            n_jobs=N_JOBS
            # risk_thresholds can be customized here if needed, otherwise defaults are used
        )

        logger.info("Starting hyperparameter optimization...")
        best_params, portfolio_performance_report, param_history_report = optimizer.run_optimization()

        if not best_params:
             logger.error("Optimization did not yield valid results. Best parameters not found.")
        else:
             logger.info("--- Optimization Results ---")
             logger.info(f"Best Parameters found:\n{json.dumps(best_params, indent=2)}")
             logger.info(f"\nBest Portfolio Performance Report:\n{portfolio_performance_report.to_string()}")
             logger.info(f"\nParameter History saved (see MLflow artifacts or CSV file). Head:\n{param_history_report.head().to_string()}")

    except Exception as e:
        logger.error(f"An error occurred during optimization: {e}", exc_info=True)
        # Attempt to end MLflow run if it was started by the optimizer
        if mlflow.active_run():
            mlflow.end_run("FAILED")

    # --- Run Sensitivity Analysis (Optional) ---
    if RUN_SENSITIVITY and optimizer and best_params:
        logger.info("\n--- Starting Sensitivity Analysis ---")
        try:
            analyzer = SensitivityAnalyzer(
                strategy_optimizer=optimizer, # Reuse optimizer for its config and evaluation cache
                base_params=best_params,
                numeric_perturbation=NUMERIC_PERTURBATION,
                num_samples_per_param=SENS_SAMPLES_PER_PARAM,
                num_random_samples=SENS_RANDOM_SAMPLES,
                parallel=True # Relies on optimizer's internal parallelization/caching
            )

            sensitivity_results_df, parameter_impact_df = analyzer.run()

            if sensitivity_results_df.empty:
                 logger.warning("Sensitivity analysis did not produce results.")
            else:
                logger.info("--- Sensitivity Analysis Results ---")
                logger.info(f"Sensitivity Results saved (see MLflow artifacts or CSV file). Head:\n{sensitivity_results_df.head().to_string()}")
                logger.info(f"\nParameter Impact Report (Correlation):\n{parameter_impact_df.to_string()}")

        except Exception as e:
            logger.error(f"An error occurred during sensitivity analysis: {e}", exc_info=True)
            if mlflow.active_run():
                 mlflow.end_run("FAILED") # End sensitivity run if it crashed

    elif RUN_SENSITIVITY and (not optimizer or not best_params):
        logger.warning("Skipping sensitivity analysis because optimization failed or produced no best parameters.")


    # Ensure any lingering run is terminated cleanly
    # Should not be necessary if 'with mlflow.start_run()' is used correctly inside modules
    # try:
    #     while mlflow.active_run():
    #         logger.info(f"Ending lingering MLflow run: {mlflow.active_run().info.run_id}")
    #         mlflow.end_run()
    # except Exception:
    #      pass # Ignore errors during cleanup

    logger.info("--- Script Finished ---")

# Parabolic SAR

In [ ]:
# trading_system/scripts/run_ao_optimization.py
"""
Main script to run hyperparameter optimization and sensitivity analysis
for the Awesome Oscillator strategy using the portfolio-based evaluation framework.
"""
import json
import logging
import os
import sys
from datetime import datetime, timedelta

import mlflow
import pandas as pd

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

try:
    from src.strategies.trend_following.parabolic_sar_strat import ParabolicSARStrategy
    from src.optimizer.strategy_optimizer import StrategyOptimizer
    from src.optimizer.sensitivity_analyzer import SensitivityAnalyzer
    from src.database.config import DatabaseConfig
    from utils.file_utils import load_tickers_from_yaml
except ImportError as e:
    print("Error importing modules. Make sure the script is run from the project root")
    print("or the 'src' directory is in the Python path.")
    print(f"Import Error: {e}")
    sys.exit(1)


In [ ]:
# single stock

# Logging Configuration
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger(__name__)

# MLflow Configuration
MLFLOW_TRACKING_URI = "file:./mlruns"  # Store MLflow data locally in ./mlruns
RUN_TIMESTAMP = datetime.now().strftime("%Y%m%d_%H%M%S")

# Data Configuration
TICKER_FILE_PATH = "../data/tickers.yml" # Path relative to project root
MAX_TICKERS = None # Limit tickers for faster testing, set to None to use all

# Backtest Period
START_DATE = (datetime.now() - timedelta(days=4*365)).strftime("%Y-%m-%d")
END_DATE = datetime.now().strftime("%Y-%m-%d")

# Optimization Settings
CV_FOLDS = 5
MAX_EVALS = 50  # Number of hyperparameter sets to evaluate
OPTIMIZATION_METRIC = 'harmonic_mean' # Portfolio metric to maximize (minus penalty)
N_JOBS = -1 # Use all available CPU cores for fold evaluation within optimizer

# Sensitivity Analysis Settings
RUN_SENSITIVITY = False # Set to False to skip sensitivity analysis
NUMERIC_PERTURBATION = 0.15 # +/- 15% for sensitivity
SENS_SAMPLES_PER_PARAM = 5
SENS_RANDOM_SAMPLES = 20

# --- Define Search Space for Awesome Oscillator ---

# Note: Hyperopt doesn't easily enforce short_period < long_period directly during sampling.
# The optimizer will evaluate invalid combinations, and they will likely fail or perform poorly.
# Strategy itself raises ValueError if short >= long during initialization.

from src.optimizer.search_space import parabolic_sar_strat_search_space

# --- Main Execution ---

if __name__ == "__main__":
    logger.info("--- Starting Awesome Oscillator Optimization Script ---")

    # Setup MLflow
    try:
        mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
        logger.info(f"MLflow tracking URI set to: {MLFLOW_TRACKING_URI}")
    except Exception as e:
        logger.error(f"Failed to set MLflow tracking URI: {e}")
        sys.exit(1)

        # Create MLflow experiment if it doesn't exist
    try:
        experiment_name = f"Parabolic_SAR_{RUN_TIMESTAMP}"
        # Check if experiment exists
        experiment = mlflow.get_experiment_by_name(experiment_name)
        if experiment is None:
            # Create new experiment
            experiment_id = mlflow.create_experiment(experiment_name)
            logger.info(f"Created new MLflow experiment: {experiment_name} with ID: {experiment_id}")
        else:
            experiment_id = experiment.experiment_id
            logger.info(f"Using existing MLflow experiment: {experiment_name} with ID: {experiment_id}")
        
        # Set the experiment for subsequent runs
        mlflow.set_experiment(experiment_name)
    except Exception as e:
        logger.error(f"Failed to create or set MLflow experiment: {e}")
        sys.exit(1)

    # Load Tickers
    try:
        tickers_to_run = load_tickers_from_yaml(TICKER_FILE_PATH, MAX_TICKERS)
    except Exception:
        logger.error("Failed to load tickers. Exiting.")
        sys.exit(1)

    # Database Config
    try:
        db_config = DatabaseConfig.default()
        # Optional: Add a check here to ensure DB connection is valid if possible
        logger.info("Database configuration loaded.")
    except Exception as e:
        logger.error(f"Failed to load database configuration: {e}")
        sys.exit(1)

    # --- Run Optimization ---
    optimizer = None
    best_params = {}
    portfolio_performance_report = pd.DataFrame()
    param_history_report = pd.DataFrame()

    logger.info(f"Initializing StrategyOptimizer for {ParabolicSARStrategy.__name__}")
    try:
        optimizer = StrategyOptimizer(
            strategy_class=ParabolicSARStrategy,
            db_config=db_config,
            search_space=parabolic_sar_strat_search_space,
            tickers=tickers_to_run,
            start_date=START_DATE,
            end_date=END_DATE,
            cv_folds=CV_FOLDS,
            max_evals=MAX_EVALS,
            optimization_metric=OPTIMIZATION_METRIC,
            run_name=f"Parabolic_SAR_{RUN_TIMESTAMP}",
            n_jobs=N_JOBS
            # risk_thresholds can be customized here if needed, otherwise defaults are used
        )

        logger.info("Starting hyperparameter optimization...")
        best_params, portfolio_performance_report, param_history_report = optimizer.run_optimization()

        if not best_params:
             logger.error("Optimization did not yield valid results. Best parameters not found.")
        else:
             logger.info("--- Optimization Results ---")
             logger.info(f"Best Parameters found:\n{json.dumps(best_params, indent=2)}")
             logger.info(f"\nBest Portfolio Performance Report:\n{portfolio_performance_report.to_string()}")
             logger.info(f"\nParameter History saved (see MLflow artifacts or CSV file). Head:\n{param_history_report.head().to_string()}")

    except Exception as e:
        logger.error(f"An error occurred during optimization: {e}", exc_info=True)
        # Attempt to end MLflow run if it was started by the optimizer
        if mlflow.active_run():
            mlflow.end_run("FAILED")

    # --- Run Sensitivity Analysis (Optional) ---
    if RUN_SENSITIVITY and optimizer and best_params:
        logger.info("\n--- Starting Sensitivity Analysis ---")
        try:
            analyzer = SensitivityAnalyzer(
                strategy_optimizer=optimizer, # Reuse optimizer for its config and evaluation cache
                base_params=best_params,
                numeric_perturbation=NUMERIC_PERTURBATION,
                num_samples_per_param=SENS_SAMPLES_PER_PARAM,
                num_random_samples=SENS_RANDOM_SAMPLES,
                parallel=True # Relies on optimizer's internal parallelization/caching
            )

            sensitivity_results_df, parameter_impact_df = analyzer.run()

            if sensitivity_results_df.empty:
                 logger.warning("Sensitivity analysis did not produce results.")
            else:
                logger.info("--- Sensitivity Analysis Results ---")
                logger.info(f"Sensitivity Results saved (see MLflow artifacts or CSV file). Head:\n{sensitivity_results_df.head().to_string()}")
                logger.info(f"\nParameter Impact Report (Correlation):\n{parameter_impact_df.to_string()}")

        except Exception as e:
            logger.error(f"An error occurred during sensitivity analysis: {e}", exc_info=True)
            if mlflow.active_run():
                 mlflow.end_run("FAILED") # End sensitivity run if it crashed

    elif RUN_SENSITIVITY and (not optimizer or not best_params):
        logger.warning("Skipping sensitivity analysis because optimization failed or produced no best parameters.")


    # Ensure any lingering run is terminated cleanly
    # Should not be necessary if 'with mlflow.start_run()' is used correctly inside modules
    # try:
    #     while mlflow.active_run():
    #         logger.info(f"Ending lingering MLflow run: {mlflow.active_run().info.run_id}")
    #         mlflow.end_run()
    # except Exception:
    #      pass # Ignore errors during cleanup

    logger.info("--- Script Finished ---")